# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps
import os
import numpy as np 
import json 
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ballina,-28.8667,153.5667,69.01,93,99,2.17,AU,1675704136
1,1,castro,-24.7911,-50.0119,78.93,49,54,5.03,BR,1675704137
2,2,ponta do sol,32.6667,-17.1000,64.71,54,7,7.40,PT,1675704138
3,3,vostok,46.4856,135.8833,-5.82,98,100,4.92,RU,1675704139
4,4,butaritari,3.0707,172.7902,79.90,83,100,17.60,KI,1675704139


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
gmaps.configure(api_key = geoapify_key)
city_data_df = city_data_df.dropna()
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
figure = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100, 
                                 point_radius = 4)

figure.add_layer(heat_layer)
# Display the map
figure


Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] < 80) 
                                    & (city_data_df["Max Temp"] > 70) 
                                    & (city_data_df["Wind Speed"] < 10) 
                                    & (city_data_df["Cloudiness"] == 0),:].dropna()
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,morgan city,29.6994,-91.2068,72.00,54,0,6.67,US,1675704162
130,130,salalah,17.0151,54.0924,75.29,73,0,2.30,OM,1675704201
131,131,cabo san lucas,22.8909,-109.9124,76.60,57,0,3.44,MX,1675704202
229,229,ixtapa,20.7000,-105.2000,72.63,68,0,5.75,MX,1675704251
262,262,karachi,24.9056,67.0822,75.02,64,0,0.00,PK,1675704269
341,341,yulara,-25.2406,130.9889,75.06,18,0,8.39,AU,1675704307
378,378,port augusta,-32.5000,137.7667,72.21,52,0,8.77,AU,1675704326
408,408,pecos,31.4229,-103.4932,71.31,44,0,3.44,US,1675704181


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
49,morgan city,US,29.6994,-91.2068,
130,salalah,OM,17.0151,54.0924,
131,cabo san lucas,MX,22.8909,-109.9124,
229,ixtapa,MX,20.7000,-105.2000,
262,karachi,PK,24.9056,67.0822,
341,yulara,AU,-25.2406,130.9889,
378,port augusta,AU,-32.5000,137.7667,
408,pecos,US,31.4229,-103.4932,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel

params = {
    "radius": 5000,
    "types": "lodging",
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["filter"] = # YOUR CODE HERE
    #params["bias"] = # YOUR CODE HERE
    params["location"] = f"{lat},{lng}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
     
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
morgan city - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
ixtapa - nearest hotel: No hotel found
karachi - nearest hotel: No hotel found
yulara - nearest hotel: No hotel found
port augusta - nearest hotel: No hotel found
pecos - nearest hotel: No hotel found


,City,Country,Lat,Lng,Hotel Name
49,morgan city,US,29.6994,-91.2068,No hotel found
130,salalah,OM,17.0151,54.0924,No hotel found
131,cabo san lucas,MX,22.8909,-109.9124,No hotel found
229,ixtapa,MX,20.7000,-105.2000,No hotel found
262,karachi,PK,24.9056,67.0822,No hotel found
341,yulara,AU,-25.2406,130.9889,No hotel found
378,port augusta,AU,-32.5000,137.7667,No hotel found
408,pecos,US,31.4229,-103.4932,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

markers = gmaps.marker_layer(locations)
# Configure the map plot
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(marker_layer)

# Display the map
figure

Figure(layout=FigureLayout(height='420px'))